<center>
    
    Inverse of Submatrices
    
    Author: Daniel Coble
    
    Status: Finished
</center>

I show an implementation of the method shown in [this](https://stats.stackexchange.com/questions/450146/updating-the-inverse-covariance-matrix-after-deleting-the-i-th-column-and-row-of) stackexchange post, which shows how to use a matix's inverse to calculate the inverse after row and column $i$ are removed.

In [107]:
import numpy as np
'''
_A_: supermatrix
_A_inv: inverse of supermatrix
rem_rows: row or rows that are removed to create the submatrix
rem_cols: column or columns that are removed to create the submatrix. Right now the code only works with rem_cols=rem_rows
'''
def invert_submatrix(_A_inv, rem_rows, rem_cols=None):
    rem_cols = rem_rows
    if(isinstance(rem_rows, int)):
        rem_rows = [rem_rows]
    if(isinstance(rem_cols, int)):
        rem_cols = [rem_cols]
    if(not type(rem_rows) is np.ndarray):
        rem_rows = np.array(rem_rows)
    if(not type(rem_cols) is np.ndarray):
        rem_cols = np.array(rem_cols)
    assert np.size(rem_rows) == np.size(rem_cols)
    k = np.size(rem_rows)
    
    a = _A_inv
    a = np.delete(_A_inv, rem_rows, axis=0)
    a = np.delete(a, rem_cols, axis=1)

    b = np.column_stack([_A_inv[:,i] for i in rem_cols])
    c = np.row_stack([_A_inv[[i],:] for i in rem_rows])
    d = np.row_stack([b[i,:] for i in rem_rows])
    b = np.delete(b, rem_rows, axis=0)
    c = np.delete(c, rem_cols, axis=1)

    return a - b@np.linalg.inv(d)@c

In [109]:
_A_ = np.random.rand(5, 5)
_A_inv = np.linalg.inv(_A_)
rem_rows = [1]

A = np.delete(np.delete(_A_, 1, axis=0), 1, axis=1)
A_inv = np.linalg.inv(A)

assert np.allclose(A_inv, invert_submatrix(_A_inv, rem_rows))